# Regression

## Data Preprocessing

### Combining Images from Eyes Defy Anemia Dataset with Hgb Project Dataset

In [1]:
# import os
# import shutil

# # --- Configuration ---
# root_dir = r"../data/Conjunctiva/Original/eyes_defy_anemia/dataset_anemia"
# output_dir = r"../data/Conjunctiva/Regression/eyes_defy_anemia_original_images"

# # Skip image files with these substrings
# skip_keywords = ["forniceal", "forniceal_palpebral", "palpebral"]

# # Valid image extensions
# valid_exts = ('.jpg', '.jpeg', '.png')

# # Ensure output directory exists
# os.makedirs(output_dir, exist_ok=True)

# # Loop over region folders
# for region in ["India", "Italy"]:
#     region_path = os.path.join(root_dir, region)
#     if not os.path.isdir(region_path):
#         continue

#     for sub_folder in os.listdir(region_path):
#         sub_folder_path = os.path.join(region_path, sub_folder)

#         # Skip files like .csv or .xlsx in the main India/Italy folder
#         if os.path.isfile(sub_folder_path):
#             continue

#         # Proceed if item is a folder
#         if os.path.isdir(sub_folder_path):
#             for filename in os.listdir(sub_folder_path):
#                 lower_name = filename.lower()
#                 if lower_name.endswith(valid_exts) and not any(keyword in lower_name for keyword in skip_keywords):
#                     src = os.path.join(sub_folder_path, filename)

#                     # Rename to {Sub_Folder}_{Region}.ext
#                     ext = os.path.splitext(filename)[1]
#                     new_filename = f"{sub_folder}_{region}{ext}"
#                     dst = os.path.join(output_dir, new_filename)

#                     # Ensure no name collision
#                     if os.path.exists(dst):
#                         print(f"Skipped duplicate: {new_filename}")
#                         continue

#                     shutil.copy2(src, dst)

# print(f"Renamed and extracted original eye images saved to: {output_dir}")


### Cropping Conjunctiva Regions from Images

In [1]:
# import os
# import cv2
# import torch
# import numpy as np
# from tqdm import tqdm
# from albumentations.pytorch import ToTensorV2
# import albumentations as A
# import segmentation_models_pytorch as smp

# # --- Paths ---
# TEST_DIR = "../data/Conjunctiva/Regression/eyes_defy_anemia_original_images"
# MASKED_SAVE_DIR = "../data/Conjunctiva/Regression/regression_dataset/Images"
# MODEL_PATH = "../models/Segmentation/conjunctiva_segmentation_unet.pth"

# # Create directory to save cropped images
# os.makedirs(CROPPED_SAVE_DIR, exist_ok=True)

# # --- Transforms ---
# val_transform = A.Compose([
#     A.Resize(512, 512),
#     A.Normalize(mean=(0.485, 0.456, 0.406),
#                 std=(0.229, 0.224, 0.225)),
#     ToTensorV2()
# ])

# # --- Load Model ---
# device = torch.device("cpu")

# # Use your custom UNet class if defined elsewhere
# model = smp.Unet(
#     encoder_name="resnet18",
#     encoder_weights="imagenet",
#     in_channels=3,
#     classes=1,  # Binary segmentation
#     activation=None  # Output will be logits
# )
# model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
# model.to(device)
# model.eval()

# # --- Process Images ---
# image_files = [f for f in os.listdir(TEST_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# for image_name in tqdm(image_files, desc="Saving precise conjunctiva regions"):
#     image_path = os.path.join(TEST_DIR, image_name)

#     # Load and preprocess image
#     orig_img = cv2.imread(image_path)
#     rgb_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
#     orig_h, orig_w = orig_img.shape[:2]

#     resized = A.Resize(512, 512)(image=rgb_img)
#     image_input = val_transform(image=resized["image"])["image"].unsqueeze(0).to(device)

#     with torch.no_grad():
#         pred = model(image_input)
#         pred_mask = torch.sigmoid(pred).squeeze().cpu().numpy()
#         binary_mask = (pred_mask > 0.5).astype(np.uint8) * 255

#     # Resize mask back to original resolution
#     binary_mask = cv2.resize(binary_mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)

#     # Find contours to create polygon mask
#     contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     mask_canvas = np.zeros_like(orig_img, dtype=np.uint8)

#     if contours:
#         # Fill the mask with white where conjunctiva is
#         cv2.drawContours(mask_canvas, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

#         # Bitwise AND to keep only conjunctiva region
#         masked = cv2.bitwise_and(orig_img, mask_canvas)

#         save_path = os.path.join(MASKED_SAVE_DIR, image_name)
#         cv2.imwrite(save_path, masked)

# print(f"Masked conjunctiva regions saved to: {MASKED_SAVE_DIR}")


In [70]:
# # Conjunctiva Box Cropping - Train Set
# import os
# import cv2
# import torch
# import numpy as np
# from tqdm import tqdm
# from albumentations.pytorch import ToTensorV2
# import albumentations as A
# import segmentation_models_pytorch as smp

# # --- Paths ---
# TEST_DIR = "../data/Conjunctiva/Regression/regression_dataset/Images"
# CROPPED_SAVE_DIR = "../data/Conjunctiva/Regression/regression_dataset/Box_Conjunctiva_Images"
# MODEL_PATH = "../models/Segmentation/conjunctiva_segmentation_unet.pth"

# # Create directory to save cropped images
# os.makedirs(CROPPED_SAVE_DIR, exist_ok=True)

# # --- Transforms ---
# val_transform = A.Compose([
#     A.Resize(512, 512),
#     A.Normalize(mean=(0.485, 0.456, 0.406),
#                 std=(0.229, 0.224, 0.225)),
#     ToTensorV2()
# ])

# # --- Load Model ---
# device = torch.device("cpu")

# model = smp.Unet(
#     encoder_name="resnet18",
#     encoder_weights="imagenet",
#     in_channels=3,
#     classes=1,  # Binary segmentation
#     activation=None  # Output will be logits
# )
# model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
# model.to(device)
# model.eval()

# # --- Process Images ---
# image_files = [f for f in os.listdir(TEST_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# for image_name in tqdm(image_files, desc="Cropping and saving Conjunctiva Boxes"):
#     image_path = os.path.join(TEST_DIR, image_name)

#     # Load and preprocess image
#     orig_img = cv2.imread(image_path)
#     rgb_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
#     orig_h, orig_w = orig_img.shape[:2]

#     resized = A.Resize(512, 512)(image=rgb_img)
#     image_input = val_transform(image=resized["image"])["image"].unsqueeze(0).to(device)

#     with torch.no_grad():
#         pred = model(image_input)
#         pred_mask = torch.sigmoid(pred).squeeze().cpu().numpy()
#         binary_mask = (pred_mask > 0.7).astype(np.uint8) * 255

#     # Resize mask back to original resolution
#     binary_mask = cv2.resize(binary_mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)

#     # Find contours and select the largest contour (box)
#     contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     if contours:
#         largest = max(contours, key=cv2.contourArea)
#         x, y, w, h = cv2.boundingRect(largest)

#         cropped_box = orig_img[y:y+h, x:x+w]
        
#         save_path = os.path.join(CROPPED_SAVE_DIR, image_name)
#         cv2.imwrite(save_path, cropped_box)

# print(f"Masked conjunctiva regions saved to: {CROPPED_SAVE_DIR}")

In [81]:
# # Conjunctiva Box Cropping - Val set
# import os
# import cv2
# import torch
# import numpy as np
# from tqdm import tqdm
# from albumentations.pytorch import ToTensorV2
# import albumentations as A
# import segmentation_models_pytorch as smp

# # --- Paths ---
# TEST_DIR = "../data/Conjunctiva/Regression/LMH_patient_data_mk2/HB Dataset"
# CROPPED_SAVE_DIR = "../data/Conjunctiva/Regression/LMH_patient_data_mk2/HB_Cropped_Conjunctiva_Validation_Set"
# MODEL_PATH = "../models/Segmentation/conjunctiva_segmentation_unet.pth"

# # Create directory to save cropped images
# os.makedirs(CROPPED_SAVE_DIR, exist_ok=True)

# # --- Transforms ---
# val_transform = A.Compose([
#     A.Resize(512, 512),
#     A.Normalize(mean=(0.485, 0.456, 0.406),
#                 std=(0.229, 0.224, 0.225)),
#     ToTensorV2()
# ])

# # --- Load Model ---
# device = torch.device("cpu")

# model = smp.Unet(
#     encoder_name="resnet18",
#     encoder_weights="imagenet",
#     in_channels=3,
#     classes=1,  # Binary segmentation
#     activation=None  # Output will be logits
# )
# model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
# model.to(device)
# model.eval()

# # --- Process Images ---
# image_files = [f for f in os.listdir(TEST_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# for image_name in tqdm(image_files, desc="Cropping + masking conjunctiva regions"):
#     image_path = os.path.join(TEST_DIR, image_name)

#     # Load and preprocess image
#     orig_img = cv2.imread(image_path)
#     rgb_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
#     orig_h, orig_w = orig_img.shape[:2]

#     resized = A.Resize(512, 512)(image=rgb_img)
#     image_input = val_transform(image=resized["image"])["image"].unsqueeze(0).to(device)

#     with torch.no_grad():
#         pred = model(image_input)
#         pred_mask = torch.sigmoid(pred).squeeze().cpu().numpy()
#         binary_mask = (pred_mask > 0.7).astype(np.uint8) * 255

#     # Resize mask back to original resolution
#     binary_mask = cv2.resize(binary_mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)

#     # Find largest contour
#     contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     if contours:
#         largest = max(contours, key=cv2.contourArea)
#         x, y, w, h = cv2.boundingRect(largest)

#         # Crop both the image and mask to the bounding box
#         cropped_img = orig_img[y:y+h, x:x+w]
#         cropped_mask = binary_mask[y:y+h, x:x+w]

#         # Apply mask inside the crop
#         mask_3d = cv2.merge([cropped_mask, cropped_mask, cropped_mask])
#         masked_crop = cv2.bitwise_and(cropped_img, mask_3d)

#         # Save final cropped+masked image
#         save_path = os.path.join(CROPPED_SAVE_DIR, image_name)
#         cv2.imwrite(save_path, masked_crop)


### Creating Metadata

In [3]:
# Clg Hb dataset metadata correction

# import pandas as pd
# import os

# # Load metadata CSV
# csv_path = "../data/Conjunctiva/Regression/Patient's Data.xlsx"
# df = pd.read_excel(csv_path)

# # Folder where images are stored
# image_folder = "../data/Conjunctiva/Regression/patient_data"  # <-- change this to your actual folder path
# image_files = os.listdir(image_folder)

# # Build mapping from Id -> [image names without .jpg]
# id_to_images = {}

# for filename in image_files:
#     if filename.endswith(".jpg"):
#         basename = os.path.splitext(filename)[0]  # "1vedant_righteye"
#         # Extract numeric Id from beginning of filename
#         numeric_id = ''.join(filter(str.isdigit, basename.split('_')[0]))
#         if numeric_id:
#             numeric_id = int(numeric_id)
#             id_to_images.setdefault(numeric_id, []).append(basename)

# # Create new dataframe with duplicated and renamed rows
# new_rows = []

# for idx, row in df.iterrows():
#     id_val = row['ID']
#     matched_images = id_to_images.get(id_val, [])
#     if matched_images:
#         for image_name in matched_images:
#             new_row = row.copy()
#             new_row['ID'] = image_name
#             new_rows.append(new_row)

# # Create updated DataFrame
# updated_df = pd.DataFrame(new_rows)

# # Save to new CSV
# updated_df.to_csv("../data/Conjunctiva/Regression/updated_metadata.csv", index=False)
# print("updated_metadata.csv has been saved.")


In [4]:
# Eyes defy anemia India Italy combined metadata
# import pandas as pd
# import os

# # --- File paths ---
# base_dir = r"A:/AI-Projects/betic-projects/hb-project/data/Conjunctiva/Original/eyes_defy_anemia/dataset_anemia"
# india_csv_path = os.path.join(base_dir, "India", "India.xlsx")   # Adjust filename if needed
# italy_csv_path = os.path.join(base_dir, "Italy", "Italy.xlsx")   # Adjust filename if needed
# output_csv_path = '../data/Conjunctiva/Regression/regression_dataset/eyes_defy_anemia_combined_metadata.csv'

# # --- Load the metadata ---
# df_india = pd.read_excel(india_csv_path)
# df_italy = pd.read_excel(italy_csv_path)

# # --- Select required columns ---
# columns_to_keep = ['Number', 'Hgb', 'Age', 'Gender']
# df_india = df_india[columns_to_keep].copy()
# df_italy = df_italy[columns_to_keep].copy()

# # --- Update the 'Number' column to include dataset origin ---
# df_india['Number'] = df_india['Number'].astype(str) + "_India"
# df_italy['Number'] = df_italy['Number'].astype(str) + "_Italy"

# # --- Combine the dataframes ---
# df_combined = pd.concat([df_india, df_italy], ignore_index=True)

# # --- Save the combined CSV ---
# df_combined.to_csv(output_csv_path, index=False)

# print(f"Combined metadata file saved at: {output_csv_path}")


In [5]:
# # Combined final metadata file.

# import pandas as pd
# import os

# # --- File paths ---
# existing_metadata_path = r"../data/Conjunctiva/Regression/regression_dataset/eyes_defy_anemia_combined_metadata.csv"
# new_metadata_xlsx_path = r"../data/Conjunctiva/Regression/updated_patients_data.csv"  # Clg patients dataset
# final_metadata_path = r"../data/Conjunctiva/Regression/regression_dataset/final_metadata.csv"

# # --- Read existing metadata CSV ---
# df_existing = pd.read_csv(existing_metadata_path)

# # --- Read new metadata XLSX ---
# df_new = pd.read_csv(new_metadata_xlsx_path)

# # --- Extract and rename relevant columns ---
# df_new = df_new[["ID", "AGE", "SEX", "HB"]].copy()
# df_new.rename(columns={
#     "ID": "Number",
#     "AGE": "Age",
#     "SEX": "Gender",
#     "HB": "Hgb"
# }, inplace=True)

# # --- Combine both datasets ---
# df_combined = pd.concat([df_existing, df_new], ignore_index=True)

# # --- Save final metadata CSV ---
# os.makedirs(os.path.dirname(final_metadata_path), exist_ok=True)
# df_combined.to_csv(final_metadata_path, index=False)

# print(f"Final metadata file saved at: {final_metadata_path}")


In [6]:
# # Dataset Splitting into train valid and test sets

# import os
# import shutil
# import pandas as pd
# import random

# # --- Configurations ---
# image_dir = r"../data/Conjunctiva/Regression/regression_dataset/Images"
# metadata_path = r"../data/Conjunctiva/Regression/regression_dataset/final_metadata.csv"
# output_root = r"../data/Conjunctiva/Regression/reg_split_dataset"

# # Create destination folders
# for split in ["train", "valid", "test"]:
#     split_dir = os.path.join(output_root, split, "images")
#     os.makedirs(split_dir, exist_ok=True)

# # Read metadata
# df = pd.read_csv(metadata_path)

# # Shuffle the metadata
# df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# # Split sizes
# train_size = 300
# valid_size = 100
# test_size = 7

# # Make the splits
# df_train = df_shuffled.iloc[:train_size].copy()
# df_valid = df_shuffled.iloc[train_size:train_size+valid_size].copy()
# df_test  = df_shuffled.iloc[train_size+valid_size:train_size+valid_size+test_size].copy()

# # Helper function to copy images to folders
# def copy_images(split_df, split_name):
#     for _, row in split_df.iterrows():
#         img_name = row['Number']  # Use actual image filename
#         src = os.path.join(image_dir, img_name)
#         dst = os.path.join(output_root, split_name, "images", img_name)
#         if os.path.exists(src):
#             shutil.copy2(src, dst)
#         else:
#             print(f"Image not found: {img_name}")

# # Copy images
# copy_images(df_train, "train")
# copy_images(df_valid, "valid")
# copy_images(df_test, "test")

# # Save metadata for each split
# df_train.to_csv(os.path.join(output_root, "train", "train_metadata.csv"), index=False)
# df_valid.to_csv(os.path.join(output_root, "valid", "valid_metadata.csv"), index=False)
# df_test.to_csv(os.path.join(output_root, "test", "test_metadata.csv"), index=False)

# print("Dataset split completed and saved.")


In [7]:
# # Improved script for dataset splitting
# # Dataset Splitting into train valid and test sets (Only existing images)
# import os
# import shutil
# import pandas as pd
# import random

# # --- Configurations ---
# image_dir = r"../data/Conjunctiva/Regression/regression_dataset/Images"
# metadata_path = r"../data/Conjunctiva/Regression/regression_dataset/final_metadata.csv"
# output_root = r"../data/Conjunctiva/Regression/reg_split_dataset"

# # Create destination folders
# for split in ["train", "valid", "test"]:
#     split_dir = os.path.join(output_root, split, "images")
#     os.makedirs(split_dir, exist_ok=True)

# # Read metadata
# df = pd.read_csv(metadata_path)

# # Filter metadata to only include existing images
# def check_image_exists(img_name):
#     # Try with .jpg extension
#     if not img_name.endswith('.jpg'):
#         img_name = img_name + '.jpg'
#     return os.path.exists(os.path.join(image_dir, img_name))

# # Filter dataframe to only include rows where images exist
# df['image_exists'] = df['Number'].apply(check_image_exists)
# df_existing = df[df['image_exists']].copy().drop('image_exists', axis=1)

# print(f"Total images in metadata: {len(df)}")
# print(f"Existing images: {len(df_existing)}")
# print(f"Missing images: {len(df) - len(df_existing)}")

# # Shuffle the existing metadata
# df_shuffled = df_existing.sample(frac=1, random_state=42).reset_index(drop=True)

# # Adjust split sizes based on available images
# total_existing = len(df_shuffled)
# if total_existing < 407:  # 300 + 100 + 7
#     print(f"Warning: Only {total_existing} images available, adjusting split sizes proportionally...")
#     train_ratio = 300 / 407
#     valid_ratio = 100 / 407
#     test_ratio = 7 / 407
    
#     train_size = int(total_existing * train_ratio)
#     valid_size = int(total_existing * valid_ratio)
#     test_size = total_existing - train_size - valid_size
# else:
#     train_size = 300
#     valid_size = 100
#     test_size = 7

# print(f"Split sizes - Train: {train_size}, Valid: {valid_size}, Test: {test_size}")

# # Make the splits
# df_train = df_shuffled.iloc[:train_size].copy()
# df_valid = df_shuffled.iloc[train_size:train_size+valid_size].copy()
# df_test  = df_shuffled.iloc[train_size+valid_size:train_size+valid_size+test_size].copy()

# # Helper function to copy images to folders
# def copy_images(split_df, split_name):
#     copied_count = 0
#     for _, row in split_df.iterrows():
#         img_name = row['Number']
        
#         # Add .jpg extension if not present
#         if not img_name.endswith('.jpg'):
#             img_name = img_name + '.jpg'
            
#         src = os.path.join(image_dir, img_name)
#         dst = os.path.join(output_root, split_name, "images", img_name)
        
#         if os.path.exists(src):
#             shutil.copy2(src, dst)
#             copied_count += 1
#         else:
#             print(f"Unexpected: Image not found during copy: {img_name}")
    
#     print(f"Copied {copied_count} images to {split_name}")
#     return copied_count

# # Copy images
# print("\nCopying images...")
# train_copied = copy_images(df_train, "train")
# valid_copied = copy_images(df_valid, "valid")
# test_copied = copy_images(df_test, "test")

# # Save metadata for each split
# df_train.to_csv(os.path.join(output_root, "train", "train_metadata.csv"), index=False)
# df_valid.to_csv(os.path.join(output_root, "valid", "valid_metadata.csv"), index=False)
# df_test.to_csv(os.path.join(output_root, "test", "test_metadata.csv"), index=False)

# print(f"\nDataset split completed!")
# print(f"Final counts - Train: {train_copied}, Valid: {valid_copied}, Test: {test_copied}")
# print(f"Total images processed: {train_copied + valid_copied + test_copied}")

In [105]:
# import os
# import pandas as pd

# # --- Paths ---
# excel_file = "../data/Conjunctiva/Regression/LMH_patient_data_mk2/PPG HB Data.xlsx"   # Path to Excel sheet
# images_dir = "../data/Conjunctiva/Regression/LMH_patient_data_mk2/HB_Cropped_Conjunctiva_Validation_Set"  # Folder containing images
# output_csv = "../data/Conjunctiva/Regression/LMH_patient_data_mk2/PPG_HB_Data_with_images.csv"

# # --- Load Excel sheet ---
# df = pd.read_excel(excel_file)

# # --- Build a map from ID -> image filenames ---
# image_map = {}
# for img in os.listdir(images_dir):
#     if img.lower().endswith(('.jpg', '.jpeg', '.png')):
#         patient_id = img.split("_", 1)[0]  # Extract ID
#         if patient_id not in image_map:
#             image_map[patient_id] = []
#         image_map[patient_id].append(str(img).removesuffix('.jpg'))  # changed

# # --- Build a new dataframe with one row per image ---
# rows = []
# for _, row in df.iterrows():
#     patient_id = str(row["ID"]).strip()
#     if patient_id in image_map:
#         for img_file in image_map[patient_id]:
#             new_row = row.copy()
#             new_row["Image_Files"] = img_file
#             rows.append(new_row)

# # Create new dataframe
# new_df = pd.DataFrame(rows)

# # Set image filenames as index
# new_df.set_index("Image_Files", inplace=True)

# # Save as CSV
# new_df.to_csv(output_csv)
# print(f"CSV saved as: {output_csv}")


## Data Loading

In [1]:
# Custom CNN 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import cv2
import numpy as np

In [2]:
class RGB2LAB_CLAHE(object):
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    def __call__(self, img):
        # PIL -> array
        img = np.array(img)
        # RGB -> LAB
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        # Split channels
        L, A, B = cv2.split(lab)
        # Apply CLAHE only on L channel
        L = self.clahe.apply(L)
        # Merge back 
        lab = cv2.merge((L, A, B))
        # Convert back to RGB (optional)
        rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
        # Ensuring valid uint8 range
        rgb = np.clip(rgb, 0, 255).astype(np.uint8)
        
        return Image.fromarray(rgb)
        

In [3]:
def compute_lab_mean_std(image_dir, clip_limit=2.0, tile_grid_size=(8,8)):
    from tqdm import tqdm
    
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    means, stds = [], []

    # loop through images
    for img_name in tqdm(os.listdir(image_dir)):
        if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(image_dir, img_name)
        img = np.array(Image.open(img_path).convert("RGB"))

        # RGB -> LAB
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)

        # CLAHE on L-channel
        L, A, B = cv2.split(lab)
        L = clahe.apply(L)
        lab = cv2.merge((L, A, B))

        # scale to [0,1] before computing mean/std
        lab = lab.astype(np.float32) / 255.0

        # per-channel mean/std
        means.append(lab.reshape(-1, 3).mean(axis=0))
        stds.append(lab.reshape(-1, 3).std(axis=0))

    # average across dataset
    mean = np.mean(means, axis=0)
    std = np.mean(stds, axis=0)

    return mean, std

In [4]:
# lab_mean, lab_std = compute_lab_mean_std(image_dir='../data/Conjunctiva/Regression/regression_dataset/Box_Conjunctiva_Images')

## Model Architecture

In [5]:
class LogCoshLoss(nn.Module):
    def __init__(self, reduction='mean'):
        super().__init__()
        self.reduction = reduction

    def forward(self, pred, target):
        x = pred - target

        x = x.clamp(min=-20, max=20)
        loss = x + torch.nn.functional.softplus(-2.0 * x) - torch.log(torch.tensor(2.0, device=x.device))

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()

        return loss

In [6]:
train_transforms = transforms.Compose([
    RGB2LAB_CLAHE(), 
    transforms.Resize((256, 256)), 
    transforms.RandomResizedCrop(224, scale=(0.9, 1.25)), 
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomRotation(15), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.289, 0.545, 0.505], std=[0.2864, 0.0456, 0.0205])
])

val_transforms = transforms.Compose([
    RGB2LAB_CLAHE(), 
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.289, 0.545, 0.505], std=[0.2864, 0.0456, 0.0205])
])

In [7]:
class SpatialAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()

        self.conv = nn.Conv2d(in_channels, 1, kernel_size=1)
        nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
        
        if self.conv.bias is not None:
            nn.init.zeros_(self.conv.bias)

    def forward(self, x):
        attn = self.conv(x)   # B, 1, H, W
        attn = torch.sigmoid(attn)  # [0, 1]

        return x * attn   # broadcast multiply
            

In [8]:
class HgbCNN(nn.Module):
    def __init__(self):
        super(HgbCNN, self).__init__()

        self.features = nn.Sequential(
            ## First Convolutional Block
            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1, dilation=2, groups=3),
            nn.GroupNorm(num_groups=1, num_channels=3), 
            nn.ReLU(),  
            
            # Second Convolutional Block
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, dilation=2), 
            nn.GroupNorm(num_groups=8, num_channels=32),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Third Convolutional Block
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, groups=32), 
            nn.GroupNorm(num_groups=8, num_channels=64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Fourth Convolutional Block
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride=1, padding=1, dilation=2), 
            nn.GroupNorm(num_groups=8, num_channels=128),
            nn.ReLU()
        )

        # Spatial Attention Block
        self.attn = SpatialAttention(in_channels=128)
            
        # Classifier Head
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

        # init
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)
        nn.init.kaiming_normal_(self.fc3.weight)
        nn.init.zeros_(self.fc3.bias)
        

    def forward(self, x):
        x = self.features(x)
        x = self.attn(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = nn.functional.relu(self.fc1(x), inplace=True)
        x = nn.functional.relu(self.fc2(x), inplace=True)
        x = self.fc3(x)

        return x
        

In [9]:
def model_summary(model):
    print('===============================================Model Summary=============================================\n')

    for name, param in model.named_parameters():
        status = 'Trainable' if param.requires_grad else 'Non-Trainable (Frozen)'
        print(f'{name:30} | {status}')

    print('\n========================================================================================================\n')

    total_params = sum(param.numel() for param in model.parameters()) 
    trainable_params = sum(param.numel() for param in model.parameters() if param.requires_grad)
    frozen_params = total_params - trainable_params

    print(f'Total Parameters: {total_params:,}')
    print(f'Trainable Parameters: {trainable_params:,}')
    print(f'Frozen Parameters: {frozen_params:,}\n')

In [10]:
class ConjunctivaDataset(Dataset):
    def __init__(self, image_dir, metadata_csv_path, transforms=None):
        self.image_dir = image_dir
        self.metadata = pd.read_csv(metadata_csv_path)
        self.transforms = transforms

        self.valid_exts = ('.jpg', '.jpeg', '.png')
        self.image_files = [img for img in os.listdir(self.image_dir) 
                            if img.lower().endswith(self.valid_exts) and os.path.splitext(img)[0] in list(self.metadata['Number'])]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img_id = os.path.splitext(img_path)[0]
        
        image = Image.open(os.path.join(self.image_dir, img_path)).convert('RGB')
        hb_val = float(self.metadata.loc[self.metadata['Number'] == img_id, 'Hgb'].values[0])

        if self.transforms:
            image = self.transforms(image)
            
        return image, hb_val

In [11]:
train_dataset = ConjunctivaDataset(image_dir='../data/Conjunctiva/Regression/regression_dataset/Box_Conjunctiva_Images', 
                                   metadata_csv_path='../data/Conjunctiva/Regression/regression_dataset/regression_complete_metadata.csv', 
                                   transforms=train_transforms)

val_dataset = ConjunctivaDataset(image_dir='../data/Conjunctiva/Regression/LMH_patient_data_mk2/HB_Cropped_Conjunctiva_Validation_Set', 
                                 metadata_csv_path='../data/Conjunctiva/Regression/LMH_patient_data_mk2/PPG_HB_Data_with_images.csv', 
                                 transforms=val_transforms)

In [12]:
hb_targets = train_dataset.metadata['Hgb'].values

In [13]:
from torch.utils.data import WeightedRandomSampler

bins = np.digitize(hb_targets, bins=np.linspace(min(hb_targets), max(hb_targets), 10))
class_sample_count = np.bincount(bins)
weights = 1. / class_sample_count[bins]
sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

In [14]:
train_loader = DataLoader(dataset=train_dataset, batch_size=24, sampler=sampler, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=24, shuffle=False)

## Model Training

In [15]:
class EarlyStopping():
    def __init__(self, patience=5, tol=0.01):
        self.patience = patience
        self.tol = tol
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.tol:
            self.counter += 1
            if self.counter > self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0          
            

In [16]:
def train_model(model, model_name, train_loader, val_loader, epochs=50, learning_rate=1e-1):
    from tqdm import tqdm
    from torch.utils.tensorboard import SummaryWriter
    
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    loss_fn = nn.SmoothL1Loss(beta=0.5)
    # loss_fn = LogCoshLoss()
    # loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
    # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, epochs=epochs, 
    #                                                 steps_per_epoch=len(train_loader), pct_start=0.1, 
    #                                                 div_factor=10, final_div_factor=100)
    
    early_stopping = EarlyStopping(patience=5)
    
    writer = SummaryWriter(log_dir=f'../reports/{model_name}')

    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        train_progress_bar = tqdm(train_loader, desc=f'Epoch: {epoch+1}/{epochs}', leave=True)

        # image, hb_val => x, y
        for x, y in train_progress_bar:
            x, y = x.to(device), y.float().unsqueeze(1).to(device)
            
            optimizer.zero_grad()
            
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()
            # scheduler.step()

            train_loss += loss.item() * x.size(0)

            train_progress_bar.set_postfix(loss=f'{loss.item():.3f}')

        train_epoch_loss = train_loss / len(train_loader.dataset)


        model.eval()
        val_loss = 0.0
        val_progress_bar = tqdm(val_loader, desc=f'Epoch: {epoch+1}/{epochs}', leave=True)
        
        with torch.no_grad():
            for x, y in val_progress_bar:
                x, y = x.to(device), y.float().unsqueeze(1).to(device)
                y_pred = model(x)
                loss = loss_fn(y_pred, y)
                val_loss += loss.item() * x.size(0)

                val_progress_bar.set_postfix(loss=f'{loss.item():.3f}')

        val_epoch_loss = val_loss / len(val_loader.dataset)

        if val_epoch_loss < best_val_loss:
            best_val_loss = val_epoch_loss
            torch.save(model.state_dict(), f'../models/{model_name}_best_v1.pt')
        
        scheduler.step(val_epoch_loss)

        writer.add_scalars('Loss', {'Train': train_epoch_loss, 'Val': val_epoch_loss}, epoch+1)
        
        print(f'Train Loss: {train_epoch_loss:.3f} | Val Loss: {val_epoch_loss:.3f}')
        
        early_stopping(val_epoch_loss)
        
        if early_stopping.early_stop:
            print('Early Stopping triggered...')
            print(f'Best Val Loss: {early_stopping.best_loss}')
            break

    writer.close()


In [17]:
import torchvision.models as models

hgbcnn = HgbCNN()
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
convnext = models.convnext_tiny(weights='IMAGENET1K_V1')

In [18]:
for param in resnet18.parameters():
    param.requires_grad = False

in_features = resnet18.fc.in_features

resnet18.fc = nn.Sequential(
    nn.Linear(in_features, out_features=256), 
    nn.ReLU(),  
    nn.Linear(256, 128),
    nn.ReLU(), 
    nn.Linear(128, 64), 
    nn.ReLU(), 
    nn.Linear(64, 1)
)

In [19]:
for param in convnext.parameters():
    param.requires_grad = False

    
convnext.classifier[2] = nn.Sequential(
    nn.Linear(768, 256), 
    nn.ReLU(), 
    nn.Linear(256, 128), 
    nn.ReLU(), 
    nn.Linear(128, 64), 
    nn.ReLU(),
    nn.Linear(64, 1)
)

In [20]:
model_summary(convnext)

===============================================Model Summary=============================================

features.0.0.weight            | Non-Trainable (Frozen)
features.0.0.bias              | Non-Trainable (Frozen)
features.0.1.weight            | Non-Trainable (Frozen)
features.0.1.bias              | Non-Trainable (Frozen)
features.1.0.layer_scale       | Non-Trainable (Frozen)
features.1.0.block.0.weight    | Non-Trainable (Frozen)
features.1.0.block.0.bias      | Non-Trainable (Frozen)
features.1.0.block.2.weight    | Non-Trainable (Frozen)
features.1.0.block.2.bias      | Non-Trainable (Frozen)
features.1.0.block.3.weight    | Non-Trainable (Frozen)
features.1.0.block.3.bias      | Non-Trainable (Frozen)
features.1.0.block.5.weight    | Non-Trainable (Frozen)
features.1.0.block.5.bias      | Non-Trainable (Frozen)
features.1.1.layer_scale       | Non-Trainable (Frozen)
features.1.1.block.0.weight    | Non-Trainable (Frozen)
features.1.1.block.0.bias      | Non-Trainable (Froze

In [21]:
model_summary(resnet18)

===============================================Model Summary=============================================

conv1.weight                   | Non-Trainable (Frozen)
bn1.weight                     | Non-Trainable (Frozen)
bn1.bias                       | Non-Trainable (Frozen)
layer1.0.conv1.weight          | Non-Trainable (Frozen)
layer1.0.bn1.weight            | Non-Trainable (Frozen)
layer1.0.bn1.bias              | Non-Trainable (Frozen)
layer1.0.conv2.weight          | Non-Trainable (Frozen)
layer1.0.bn2.weight            | Non-Trainable (Frozen)
layer1.0.bn2.bias              | Non-Trainable (Frozen)
layer1.1.conv1.weight          | Non-Trainable (Frozen)
layer1.1.bn1.weight            | Non-Trainable (Frozen)
layer1.1.bn1.bias              | Non-Trainable (Frozen)
layer1.1.conv2.weight          | Non-Trainable (Frozen)
layer1.1.bn2.weight            | Non-Trainable (Frozen)
layer1.1.bn2.bias              | Non-Trainable (Frozen)
layer2.0.conv1.weight          | Non-Trainable (Froze

In [22]:
model_summary(hgbcnn)

===============================================Model Summary=============================================

features.0.weight              | Trainable
features.0.bias                | Trainable
features.1.weight              | Trainable
features.1.bias                | Trainable
features.3.weight              | Trainable
features.3.bias                | Trainable
features.4.weight              | Trainable
features.4.bias                | Trainable
features.7.weight              | Trainable
features.7.bias                | Trainable
features.8.weight              | Trainable
features.8.bias                | Trainable
features.11.weight             | Trainable
features.11.bias               | Trainable
features.12.weight             | Trainable
features.12.bias               | Trainable
attn.conv.weight               | Trainable
attn.conv.bias                 | Trainable
fc1.weight                     | Trainable
fc1.bias                       | Trainable
fc2.weight                     | 

In [23]:
# train_model(hgbcnn, 'HgbCNN', train_loader, val_loader, epochs=30)

Epoch: 1/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.36it/s, loss=8.687]


Train Loss: 38.997 | Val Loss: 8.242


Epoch: 2/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.56it/s, loss=1.736]


Train Loss: 3.285 | Val Loss: 1.534


Epoch: 3/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.44it/s, loss=1.353]


...

Train Loss: 1.976 | Val Loss: 1.283


Epoch: 11/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.05it/s, loss=1.346]


Train Loss: 1.883 | Val Loss: 1.472


Epoch: 12/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.65it/s, loss=1.404]

Train Loss: 1.999 | Val Loss: 1.280
Early Stopping triggered...
Best Val Loss: 1.2764017324189882


In [24]:
# train_model(resnet18, 'ResNet18', train_loader, val_loader, epochs=30)

Epoch: 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.30it/s, loss=11.229]


Train Loss: 562.754 | Val Loss: 10.788


Epoch: 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.39it/s, loss=10.798]


Train Loss: 13.158 | Val Loss: 10.352


Epoch: 3/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.34it/s, loss=9.274]


...

Train Loss: 1.845 | Val Loss: 1.265


Epoch: 20/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.34it/s, loss=1.377]


Train Loss: 1.946 | Val Loss: 1.268


Epoch: 21/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.31it/s, loss=1.401]

Train Loss: 1.943 | Val Loss: 1.278
Early Stopping triggered...
Best Val Loss: 1.2692934564642004


In [25]:
# train_model(convnext, 'ConvNeXt', train_loader, val_loader, epochs=30)

Epoch: 1/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.16s/it, loss=4.574]


Train Loss: 176.617 | Val Loss: 4.274


Epoch: 2/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.16s/it, loss=3.257]


Train Loss: 2.933 | Val Loss: 3.052


Epoch: 3/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.15s/it, loss=3.425]


...

Train Loss: 2.225 | Val Loss: 1.774


Epoch: 10/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.19s/it, loss=1.832]


Train Loss: 1.460 | Val Loss: 1.750


Epoch: 11/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.40s/it, loss=1.747]

Train Loss: 1.342 | Val Loss: 1.736
Early Stopping triggered...
Best Val Loss: 1.6403366359504494


## Model Testing

In [48]:
import os
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator

In [49]:
def load_tensorboard_scalars(log_dir, tag='loss/train'):
    ea = event_accumulator.EventAccumulator(log_dir)
    ea.Reload()

    return [x.value for x in ea.Scalars(tag)]


In [50]:
runs = {
    'HgbCNN': ('../reports/HgbCNN/Loss_Train', '../reports/HgbCNN/Loss_Val'), 
    'ResNet18': ('../reports/ResNet18/Loss_Train', '../reports/ResNet18/Loss_Val'), 
    'ConvNeXt': ('../reports/ConvNeXt/Loss_Train', '../reports/ConvNeXt/Loss_Val')
}

loss_curves = {}

for model, path in runs.items():
    train_loss = load_tensorboard_scalars(path[0], 'Loss')
    val_loss = load_tensorboard_scalars(path[1], 'Loss')
    loss_curves[model] = (train_loss, val_loss)

In [56]:
# plt.figure(figsize=(10,6))

# for model, (train, val) in loss_curves.items():
#     plt.plot(train, label=f'{model} Train Loss', linestyle='--')
#     plt.plot(val, label=f'{model} Val Loss')

# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Training vs Validation Loss (hgbCNN vs ResNet18 vs ConvNeXt')
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.savefig('../reports/HgbCNN_vs_ResNet18_vs_ConvNeXt')

In [54]:
# for model, (train, val) in loss_curves.items():
#     plt.plot(train, label=f'{model} Train Loss', linestyle='--')
#     plt.plot(val, label=f'{model} Val Loss')

#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.title(f'{model}: Training vs Validation Loss')
#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.savefig(f'../reports/{model}_Loss_Plot')
#     plt.clf()

In [57]:
# cmd to view loss plots in tensorboard web server
# tensorboard --logdir=../reports --port=6006